<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/kaggle/04-titanic-machine-learning-from-disaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### kaggle 에서 가져왔습니다.

# Titanic - Machine Learning from Diaster

In [ ]:
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# kaggle에서 titanic 데이터 셋 가져오기

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c titanic

In [ ]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Going to use these 5 base models for the stacking
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold

# Feature Exploration, Engineering and Cleaning

In [ ]:
# Load in the train and test datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
# Store our passenger ID for easy access
PassengerId = test['PassengerId']

In [ ]:
train.head(3)

In [ ]:
full_data = [train, test]

In [ ]:
# Some features of my own that I have added in
# Gives the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)

In [ ]:
# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train['Cabin'].apply(lambda x: 0 if pd.isnull(x) else 1)
test['Has_Cabin'] = test['Cabin'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [ ]:
# Feature engineering steps taken from Sina
# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

In [ ]:
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

In [ ]:
# Remove all NULLS in the Embarked column
for dataset in full_data:
  dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [ ]:
# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
  dataset['Fare'] = dataset['Fare'].fillna(dataset['Fare'].median())

train['CategoricalFare'] = pd.qcut(train['Fare'], 4)

In [ ]:
# Create a New feature CategoricalAge
for dataset in full_data:
  age_avg = dataset['Age'].mean()
  age_std = dataset['Age'].std()
  age_null_count = dataset['Age'].isnull().sum()
  age_null_random_list = np.random.randint(age_avg-age_std, age_avg+age_std, size=age_null_count, dtype=int)
  dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
  dataset['Age'] = dataset['Age'].astype(int)
  
train['CategoricalAge'] = pd.qcut(train['Age'], 5)